In [1]:
from paddleocr import PaddleOCR,draw_ocr
from PIL import Image
from IPython.display import display

In [2]:
ocr = PaddleOCR(use_angle_cls=True, lang='en', show_log = False)
img_path = 'input_2.jpg'
result = ocr.ocr(img_path, cls=True)
ocr_text = ''
for res in result:
    for line in res:
        ocr_text += (line[-1][0] + '\n')

print(ocr_text)

Santa Monica Police Officers
&Firefighters'Association
POST OFFICE BOX2160 SANTA MONICA,CALIFORNIA
90407-2160
Maps and Gov Info
RESERVE
Campaign Literature
CAMPAIGNS,POLITICAL-1996
MUNICIPAL, SANTA.MONICA
GENERAL
Dear
Your police officers, paramedics, firefighters and our most unique City need your help on Tuesday,
November 5th.Your commitment to vote insures that our elected officials will continue to follow the course
of action that you feel is in Santa Monica's best interest.
There have been a lot of positive changes over the past two years: Palisades Park, 3rd Street Promenade,
Lincoln Park,Santa Monica Pier and the homeless issues. It is important that we not slip backwards. You
can make an important difference in these policy making decisions by casting your vote for the council
candidates that are committed to keeping your safety and protection their top priorities.
Members of both the fire and police associations have interviewed the council candidates and we have
endorsed the 

In [3]:
from refined.data_types.base_types import Span, Entity
from refined.inference.processor import Refined

In [ ]:
refined = Refined.from_pretrained(model_name='wikipedia_model',
                                  entity_set='wikipedia',
                                  use_precomputed_descriptions=False)

In [ ]:
text = "ocr_text"
spans = refined.process_text(text)

named_entities = []
for span in spans:
    print(f"{span}\n")
    if isinstance(span[1], Entity):
        url = construct_wikidata_url(span[1].wikidata_entity_id)
        name = span[0]
        named_entities.append((name, url))
        

def construct_wikidata_url(id):
    return "https://www.wikidata.org/wiki/" + id

In [ ]:
openai_api_key = ""

prompt = (
        "Below is the OCR text from a document from UCLA's Campaign Literature collection, "
        " along with a list of named entities known to be associated with the document. "
        "Provide a Dublin Core metadata record for the original document. "
        "At a minumum, include 'Title', 'Subject', and 'Description' fields. "
        "Use 'sentence case' for the 'Title' field. "
        "Return only Dublin Core fields, correctly formatted as JSON. "
        f"The named entities are enclosed in three curly braces below: {{{{spans}}}}"
        f"The text from the document is enclosed in three single quotes below: '''{ocr_text}'''"
    )

from pprint import pprint

In [ ]:
if openai_api_key:
    openai.api_key = openai_api_key
    response = get_gpt_as_dict(prompt)
    pprint(response)

else:
    print("Copy and paste the below prompt into your chosen LLM:")
    pprint(prompt)
    
def get_gpt_as_dict(prompt: str, temperature: float = 0.1, top_p: float = 0.3) -> dict:
    completion = openai.ChatCompletion.create(
        model="gpt-4-turbo",
        messages=[
            {
                "role": "user",
                "content": prompt,
            },
        ],
        temperature=temperature,
        top_p=top_p,
    )
    response = completion.choices[0].message.content
    return json.loads(response)